In [46]:
from lxml import etree
import re
from collections import defaultdict

In [47]:
xml_file = "/Users/koichiito/Downloads/j.ijcard.2017.02.xml"
root = etree.parse(xml_file).getroot()
__payload_type = u'full-text-retrieval-response'
elements = root.xpath(f"//*[translate(name(), 'FULLTEXTR', 'fulltextr')='{__payload_type}']")
if elements:
    print("Element found")
else:
    print("Element not found")



Element found


In [59]:
head = root.xpath(f"//*[translate(name(), 'FULLTEXTR', 'fulltextr')='head']")[0]
keywords = [etree.tostring(keyword, method="text", encoding="unicode").replace("\n", " ").strip() for keyword in head.xpath(".//ce:keyword", namespaces={"ce": ns["ce"]})]
print(keywords)

['MicroRNAs', 'Biomarkers', 'Chronic heart failure', 'Prognostication', 'Mortality']


In [66]:
abstract = "".join(head.xpath(".//ce:abstract//ce:simple-para/text()", namespaces={"ce":ns["ce"]}))
abstract

'Several studies have suggested circulating microRNAs (miRs) are associated with heart failure, but these studies were small, and limited to single miR measurements.We examined 7 miRs which were previously linked to heart failure, and tested whether their temporal expression level predicts prognosis in a prospective cohort of chronic heart failure (CHF) patients.In 2011–2013, 263 CHF patients were included. At inclusion and subsequently every 3months, we measured 7miRs. The primary endpoint (PE) comprised heart failure hospitalization, cardiovascular mortality, cardiac transplantation and LVAD implantation. Associations between temporal miR patterns and the PE were investigated by joint modelling, which combines mixed models with Cox regression.Mean age was 67±13years, 72% were men and 27% NYHA classes III–IV. We obtained 873 blood samples (median 3 [IQR 2–5] per patient). The PE was reached in 41 patients (16%) during a median follow-up of 0.9 [0.6–1.4] years. The temporal pattern of 

In [ ]:
elements_section = elements[0].findall("ce:section", namespaces={"ce": 'http://www.elsevier.com/xml/common/dtd'})  

In [ ]:
elements_section[2].text

'\n                  '

In [ ]:
elements_section

[<Element {http://www.elsevier.com/xml/common/dtd}section at 0x10ddca900>,
 <Element {http://www.elsevier.com/xml/common/dtd}section at 0x10de4fc80>,
 <Element {http://www.elsevier.com/xml/common/dtd}section at 0x10de4fe80>,
 <Element {http://www.elsevier.com/xml/common/dtd}section at 0x10de4fe40>,
 <Element {http://www.elsevier.com/xml/common/dtd}section at 0x11204c6c0>,
 <Element {http://www.elsevier.com/xml/common/dtd}section at 0x11204c680>]

In [8]:
ns = root.nsmap
ns

{None: 'http://www.elsevier.com/xml/svapi/article/dtd',
 'bk': 'http://www.elsevier.com/xml/bk/dtd',
 'cals': 'http://www.elsevier.com/xml/common/cals/dtd',
 'ce': 'http://www.elsevier.com/xml/common/dtd',
 'ja': 'http://www.elsevier.com/xml/ja/dtd',
 'mml': 'http://www.w3.org/1998/Math/MathML',
 'sa': 'http://www.elsevier.com/xml/common/struct-aff/dtd',
 'sb': 'http://www.elsevier.com/xml/common/struct-bib/dtd',
 'tb': 'http://www.elsevier.com/xml/common/table/dtd',
 'xlink': 'http://www.w3.org/1999/xlink',
 'xocs': 'http://www.elsevier.com/xml/xocs/dtd',
 'dc': 'http://purl.org/dc/elements/1.1/',
 'dcterms': 'http://purl.org/dc/terms/',
 'prism': 'http://prismstandard.org/namespaces/basic/2.0/',
 'xsi': 'http://www.w3.org/2001/XMLSchema-instance'}

In [9]:
coredata = root.find("coredata", namespaces={None:"http://www.elsevier.com/xml/svapi/article/dtd"})
doi = coredata.find("prism:doi", namespaces={"prism":ns["prism"]}).text
print(doi)

10.1016/j.ijcard.2017.02.078


In [10]:
ns[None]

'http://www.elsevier.com/xml/svapi/article/dtd'

In [ ]:
# find sections
body = root.xpath(f"//*[translate(name(), 'FULLTEXTR', 'fulltextr')='body']")
for i in body[0].iter():
    print(i.text)

In [76]:
# find sections
# sections_element_root = root.xpath(f"//*[translate(name(), 'FULLTEXTR', 'fulltextr')='ce:sections']")
# find sections
body = root.xpath(f"//*[translate(name(), 'FULLTEXTR', 'fulltextr')='body']")
# for i in sections_element_root[0].iter():
#     print(i.text)

# get a list of section 
section_element_root = body[0].find("ce:sections", namespaces = {"ce":ns["ce"]}).findall("ce:section", namespaces={"ce": ns["ce"]})  

for section in section_element_root:
    section_title = section.find("ce:section-title",namespaces={"ce": ns["ce"]}).text
    # print(section_title.text)
    section_below_list = section.findall("ce:section",namespaces={"ce": ns["ce"]})
    if len(section_below_list) > 0:
        for section_below in section_below_list:
            print("-"*10)
            section_title = section_below.find("ce:section-title",namespaces={"ce": ns["ce"]}).text
            paragraphs = [re.sub(r'\[.*?\]', '', re.sub(" +", " ", etree.tostring(para, method="text", encoding="unicode").\
                replace("\n", " "))).replace(" .", ".").strip() for para in section_below.xpath(".//ce:para", namespaces={"ce": ns["ce"]})]
            print(section_title, paragraphs)
            print("-"*10)
    else:
        print("*"*10)
        # section_title = section_below_list[0].find("ce:section",namespaces={"ce": ns["ce"]})
        paragraphs = [re.sub(r'\[.*?\]', '', re.sub(" +", " ", etree.tostring(para, method="text", encoding="unicode").\
            replace("\n", " "))).replace(" .", ".").strip() for para in section.xpath(".//ce:para", namespaces={"ce": ns["ce"]})]
        print(section_title, paragraphs)
        print("*"*10)
    # section_text_list = section.xpath(".//ce:section-title/text() | .//ce:para/text()", namespaces={"ce": ns["ce"]})
    # print(section_text_list)
    # print("-"*10)

**********
Introduction ['Contemporary treatment of chronic heart failure ((C)HF) generally aims to stabilize or at least decelerate disease progression. Adjustment of pharmacotherapy is largely based on clinical judgement and thus mostly relies on the worsening of symptoms or signs. However, in the context of the complexity of HF therapy, considerable clinical skills, as well as cooperation of the patients, are required to recognize opportunities to titrate therapies and to implement such changes intervene early and timely. Consequently, higher-risk patients may be undertreated. Blood biomarkers have the potential to monitor subtle changes in the heart that reflect and possibly predict adverse changes before they become clinically apparent or reported. The value of biomarkers, such as B-type natriuretic peptides (BNP), cardiac troponins and C-reactive protein (CRP), for risk stratification of CHF patients has already been demonstrated. Moreover, natriuretic peptide-guided HF therapy h

In [203]:
for i in section_element_root:
    print(i.tag)
    # print("-"*10)

{http://www.elsevier.com/xml/common/dtd}section
{http://www.elsevier.com/xml/common/dtd}section
{http://www.elsevier.com/xml/common/dtd}section
{http://www.elsevier.com/xml/common/dtd}section
{http://www.elsevier.com/xml/common/dtd}section
{http://www.elsevier.com/xml/common/dtd}section


In [ ]:
for i in sections_element_root[0].iter():
    print(i.text)
    

In [ ]:
# check json file
import json
with open("data/raw/papers.json", "r") as f:
    data = json.load(f)

print(data["10.1016/j.scs.2022.104272"]["null"])

In [73]:
def categorize_section(title):
    """a method to categorize titles

    Args:
        title (_type_): _description_
    """
    patterns = [
        (r'.*intro.*|.*background.*|.*problem statement.*|.*research objective.*', 'Introduction'),
        (r'.*review.*|.*work.*', 'Literature review'),
        (r'(?!.*result.*)(.*method.*|.*data.*|.*experiment.*|.*model.*|.*pipeline.*|.*evaluation.*|.*design.*|.*materials.*)', 'Methodology'),
        (r'.*result.*|.*discussion.*|.*conclusion.*', 'Results')
        ]

    title = title.lower()  # convert to lowercase
    for pattern, label in patterns:
        if re.search(pattern, title):
            return(label)
    # if it doesn't get caught by any labels, then return as "others"
    return("Others")

categorize_section("research statement")

'Others'

In [1]:
string_total = ""
for i in ["a","b","c"]:
    string_total += i
print(string_total)

abc
